In [ ]:
import torch
import segmentation_models_pytorch as smp
# https://chat.openai.com/share/5494b0e3-3a65-4bf3-9581-4eb52dbffe3f
class ThreeDConvolutionBlock(torch.nn.Sequential):
    def __init__(
        self, input_channels: int, output_channels: int, kernel_size: tuple[int, int, int], padding: tuple[int, int, int]
    ):
        super().__init__(
            torch.nn.Conv3d(input_channels, output_channels, kernel_size, padding=padding, padding_mode="replicate"),
            torch.nn.BatchNorm3d(output_channels),
            torch.nn.LeakyReLU(),
        )

class TwoPointFiveDSegmentor(torch.nn.Module):
    def __init__(self, ...):
        super().__init__()
        self.F = 3
        self.unet_backbone = smp.Unet(...)
        
        # エンコーダの出力チャンネル用の3D畳み込みブロックを作成（最初のチャンネルを除く）。
        three_d_conv_blocks = [
            torch.nn.Sequential(
                ThreeDConvolutionBlock(channel, channel, (2, 3, 3), (0, 1, 1)), 
                ThreeDConvolutionBlock(channel, channel, (2, 3, 3), (0, 1, 1))
            )
            for channel in self.unet_backbone.encoder.out_channels[1:]
        ]
        self.three_d_convs = torch.nn.ModuleList(three_d_conv_blocks)

    def convert_to_2d(self, three_d_conv_block: torch.nn.Module, feature_map: torch.Tensor) -> torch.Tensor:
        BxF, C, H, W = feature_map.shape
        feature_3d = feature_map.reshape(BxF // self.F, self.F, C, H, W)
        feature_3d_transposed = feature_3d.transpose(1, 2) # (B, F, C, H, W)
        output = three_d_conv_block(feature_3d_transposed)#(B, F-2, H, W) 
        output = output.squeeze(2) #(B, C, H, W) 
        return output 

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        B, C, F, H, W = input_tensor.shape
        
        # U-Netのバックボーンに渡すための入力テンソルの形状を変更。
        # 入力テンソルの形状は (B, C, F, H, W) です。
        # transpose(1, 2) を使用して、channels と F の次元を入れ替えることでテンソルの形状を (B, F, C, H, W) に変更します。
        # 次に、reshape関数を使って、バッチの次元とフレームの次元を結合します。これにより、テンソルの形状は (B * F, C, H, W) に変更されます。
        reshaped_input = input_tensor.transpose(1, 2).reshape(B * F, C, H, W)

        # 入力テンソルの形状がU-Netと互換性があることを確認。
        self.unet_backbone.check_input_shape(reshaped_input)

        # 入力テンソルをU-Netエンコーダに渡す。
        encoder_features = self.unet_backbone.encoder(reshaped_input)

        # エンコーダからの3D特徴をThreeDConvolutionBlockなどを通して2Dに変換。
        encoder_features[1:] = [self.convert_to_2d(three_d_conv, feature) for three_d_conv, feature in zip(self.three_d_convs, encoder_features[1:])]

        # 変換された特徴をU-Netデコーダに渡す。
        decoder_output = self.unet_backbone.decoder(*encoder_features)

        # U-Netのセグメンテーションヘッドを使用してセグメンテーションマスクを生成。
        segmentation_masks = self.unet_backbone.segmentation_head(decoder_output)
        
        return segmentation_masks